

파인튜닝 실습을 시작하기 위한 환경이 거의 준비된 상태입니다. 다음은 필요한 추가 작업 및 단계별 가이드를 제공합니다.

---

## 1. **필요한 라이브러리 및 도구 설치**
파인튜닝을 위해 몇 가지 Python 라이브러리를 설치해야 합니다. Hugging Face의 `transformers`, `datasets`, `accelerate` 패키지가 유용합니다.

### Miniconda 환경 생성
```bash
conda create -n llm-finetuning python=3.9 -y
conda activate llm-finetuning
```

### 패키지 설치
```bash
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
pip install transformers datasets accelerate scipy
```

---

## 2. **데이터 준비**
파인튜닝할 데이터를 준비해야 합니다. Hugging Face의 `datasets` 라이브러리를 사용하면 다양한 공개 데이터셋을 쉽게 사용할 수 있습니다.

예: IMDB 감성 분석 데이터셋 로드
```python
from datasets import load_dataset

dataset = load_dataset("imdb")
print(dataset)
```

또는, 자신만의 텍스트 데이터셋을 CSV 또는 JSON 포맷으로 준비할 수 있습니다.

---

## 3. **사전학습 모델 로드**
Hugging Face의 `transformers`를 이용해 사전 학습된 언어 모델(예: BERT, GPT-2, T5)을 로드합니다.

예: BERT 모델 로드
```python
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)
```

---

## 4. **데이터 전처리**
모델 입력에 맞게 데이터를 전처리합니다.

```python
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
```

---

## 5. **파인튜닝**
Hugging Face의 `Trainer` API를 사용하면 파인튜닝을 쉽게 수행할 수 있습니다.

### TrainingArguments 및 Trainer 설정
```python
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)
```

### 파인튜닝 실행
```python
trainer.train()
```

---

## 6. **결과 저장**
파인튜닝된 모델을 저장합니다.
```python
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")
```

---

## 7. **추론**
새로운 입력 데이터에 대해 모델 추론을 수행합니다.
```python
inputs = tokenizer("이 영화는 정말 재미있어요!", return_tensors="pt")
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)
print(predictions)
```

---

## 추가 필요 사항
- **데이터셋**: 학습에 사용할 데이터가 없다면 Hugging Face `datasets`에서 공개 데이터셋을 활용하거나 직접 준비해야 합니다.
- **GPU 사용 확인**: GPU 활용이 제대로 설정되었는지 확인하세요.
  ```bash
  nvidia-smi
  ```
- **문제 정의**: 감정 분석, 번역, 요약 등 실습 목적에 맞는 문제를 구체화하세요.

추가로 궁금한 점이 있거나 특정 문제에 대해 도움을 받고 싶다면 알려주세요! 😊

In [ ]:
# Python : 3.9.21
# Created: Jan. 03. 2025
# Updated: Jan. 03. 2025
# Author: D.W. SHIN

In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [4]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)


/home/shin/miniconda3/envs/llm-finetuning/lib/python3.9/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_17403/599353726.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [5]:
trainer.train()


  0%|          | 0/9375 [00:00<?, ?it/s]

{'loss': 0.3802, 'grad_norm': 19.89314842224121, 'learning_rate': 1.8933333333333334e-05, 'epoch': 0.16}
{'loss': 0.3016, 'grad_norm': 20.777158737182617, 'learning_rate': 1.7866666666666666e-05, 'epoch': 0.32}
{'loss': 0.2932, 'grad_norm': 11.884294509887695, 'learning_rate': 1.6800000000000002e-05, 'epoch': 0.48}
{'loss': 0.268, 'grad_norm': 24.096893310546875, 'learning_rate': 1.5733333333333334e-05, 'epoch': 0.64}
{'loss': 0.2636, 'grad_norm': 8.69232177734375, 'learning_rate': 1.4666666666666666e-05, 'epoch': 0.8}
{'loss': 0.2595, 'grad_norm': 42.78128433227539, 'learning_rate': 1.3600000000000002e-05, 'epoch': 0.96}


  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.27073293924331665, 'eval_runtime': 91.7904, 'eval_samples_per_second': 272.36, 'eval_steps_per_second': 34.045, 'epoch': 1.0}
{'loss': 0.1919, 'grad_norm': 0.05162442848086357, 'learning_rate': 1.2533333333333336e-05, 'epoch': 1.12}
{'loss': 0.1587, 'grad_norm': 0.5783259868621826, 'learning_rate': 1.1466666666666668e-05, 'epoch': 1.28}
{'loss': 0.1493, 'grad_norm': 1.3036930561065674, 'learning_rate': 1.04e-05, 'epoch': 1.44}
{'loss': 0.1437, 'grad_norm': 0.04310528561472893, 'learning_rate': 9.333333333333334e-06, 'epoch': 1.6}
{'loss': 0.1689, 'grad_norm': 44.25569534301758, 'learning_rate': 8.266666666666667e-06, 'epoch': 1.76}
{'loss': 0.1414, 'grad_norm': 0.10130887478590012, 'learning_rate': 7.2000000000000005e-06, 'epoch': 1.92}


  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.27662393450737, 'eval_runtime': 91.8337, 'eval_samples_per_second': 272.231, 'eval_steps_per_second': 34.029, 'epoch': 2.0}
{'loss': 0.1283, 'grad_norm': 0.09721854329109192, 'learning_rate': 6.133333333333334e-06, 'epoch': 2.08}
{'loss': 0.071, 'grad_norm': 0.024210352450609207, 'learning_rate': 5.0666666666666676e-06, 'epoch': 2.24}
{'loss': 0.0623, 'grad_norm': 0.10222867876291275, 'learning_rate': 4.000000000000001e-06, 'epoch': 2.4}
{'loss': 0.0743, 'grad_norm': 0.0219736248254776, 'learning_rate': 2.9333333333333338e-06, 'epoch': 2.56}
{'loss': 0.0793, 'grad_norm': 0.0652238056063652, 'learning_rate': 1.8666666666666669e-06, 'epoch': 2.72}
{'loss': 0.0722, 'grad_norm': 59.541385650634766, 'learning_rate': 8.000000000000001e-07, 'epoch': 2.88}


  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.3095468580722809, 'eval_runtime': 91.5513, 'eval_samples_per_second': 273.071, 'eval_steps_per_second': 34.134, 'epoch': 3.0}
{'train_runtime': 1229.9229, 'train_samples_per_second': 60.979, 'train_steps_per_second': 7.622, 'train_loss': 0.1736964552815755, 'epoch': 3.0}


TrainOutput(global_step=9375, training_loss=0.1736964552815755, metrics={'train_runtime': 1229.9229, 'train_samples_per_second': 60.979, 'train_steps_per_second': 7.622, 'total_flos': 1.9733329152e+16, 'train_loss': 0.1736964552815755, 'epoch': 3.0})

In [6]:
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")


('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/vocab.txt',
 './finetuned_model/added_tokens.json',
 './finetuned_model/tokenizer.json')

In [9]:
import torch

# 모델을 GPU(CUDA)로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 입력 데이터도 동일한 디바이스로 이동
inputs = tokenizer("This movie was fantastic!", return_tensors="pt").to(device)

# 모델 예측 수행
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)
print(predictions)


tensor([1], device='cuda:0')
